In [1]:
import pandas as pd 

In [3]:
people = pd.read_csv("EHRShot/sampled_person.csv")
deaths = pd.read_csv("EHRShot/sampled_death.csv")
sites = pd.read_csv("EHRShot/care_site.csv")
conditions = pd.read_csv("EHRShot/sampled_condition_occurrence.csv")

C:\Users\patri\AppData\Local\Temp\ipykernel_2172\1278085068.py:4: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  conditions = pd.read_csv("EHRShot/sampled_condition_occurrence.csv")


In [4]:
# use regex to match ICU/"intensive care"
pattern = r'\bICU\b|intensive care'
mask = sites['care_site_name'].fillna('').str.contains(pattern, case=False, regex=True)
icus = sites.loc[mask].copy()

icus.head()

,care_site_id,care_site_name,place_of_service_concept_id,location_id,care_site_source_value,place_of_service_source_value,trace_id,unit_id,load_table_id
70,529546,ICU,8756,8646458,110100006,On Campus - Outpatient Hospital,NaN,NaN,shc_clarity_dep
71,529617,INTENSIVE CARE,8756,11352401,20026201,On Campus - Outpatient Hospital,NaN,NaN,shc_clarity_dep
213,529578,ICU,8756,10178563,110100002,On Campus - Outpatient Hospital,NaN,NaN,shc_clarity_dep
451,527995,PEDIATRIC INTENSIVE CARE,0,11101381,9991085,NaN,NaN,NaN,shc_clarity_dep
508,527283,INTENSIVE CARE,0,10537544,10201253,NaN,NaN,NaN,lpch_clarity_dep


In [5]:
# find persons whose care_site is in the ICUs list
#! not working
mask = people['care_site_id'].isin(set(icus['care_site_id']))
people_in_icus = people.loc[mask].copy()

print(f'People total: {len(people)}, People in ICUs: {mask.sum()}')

People total: 2000, People in ICUs: 0


In [10]:
# find conditions where the person has an associated record in deaths
mask = conditions['person_id'].isin(set(deaths['person_id']))
conditions_with_deaths = conditions.loc[mask].copy()
conditions_with_deaths

print(f'Conditions total: {len(set(conditions['condition_concept_id']))}, Conditions with deaths: {len(set(conditions_with_deaths['condition_concept_id']))}')
print(f'People total: {len(people)}, People suffering death: {len(set(conditions_with_deaths["person_id"]))}')

Conditions total: 6376, Conditions with deaths: 3120
People total: 2000, People suffering death: 207


In [ ]:
# find people who died in their 20s
# left join people and their birth date to deaths
people_deaths = pd.merge(deaths, people.filter(['person_id', 'year_of_birth']), on='person_id', how='left')
# calculate age at death column
people_deaths['age_at_death'] = people_deaths['death_DATE'].str.slice(0,4).astype(int) - people_deaths['year_of_birth']
people_deaths[people_deaths['age_at_death'].between(20,29)]

,person_id,death_DATE,death_DATETIME,death_type_concept_id,cause_concept_id,cause_source_value,cause_source_concept_id,trace_id,unit_id,load_table_id,_death_date_external,year_of_birth,age_at_death
76,115968882,2014-07-22,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1991,23
82,115973201,2015-02-21,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1988,27
85,115967423,2015-05-12,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1990,25
110,115968171,2017-03-12,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1988,29
114,115972356,2017-08-17,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1993,24
149,115967241,2020-03-20,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1993,27
201,115972311,2022-10-02,NaN,32817,0,NaN,0,NaN,NaN,shc_patient,NaN,1995,27
